In [ ]:
#! /usr/bin/python
# -*- coding: utf-8 -*-
# @author izhangxm
# @date 2021/10/12
# @fileName train.py
# Copyright 2017 izhangxm@gmail.com. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from itertools import product
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")
from core_lib import MyDataset, get_target, r2_loss, get_predict_ks

In [3]:
def get_model(dataset, k_kinetics, k_sigma_priors = 0.01):
    # 定义参数优化模型
    mcmc_model = pm.Model()
    ## 参数个数
    params_n = 11

    # 参数K的先验分布参数
    mu_priors = 0

    ks = []
    with mcmc_model:
        for ki in range(1, params_n+1):
            p_dense = pm.HalfNormal(f"k{ki}", sigma=k_sigma_priors)
            # p_dense = pm.Normal(f"k{ki}",mu=0, sigma=sigma_priors)
            ks.append(p_dense)
    
    df = dataset.get_df()
    errors = dataset.get_errors()
    rates = dataset.get_rates()
    
    
    target= get_target(ks, df, k_kinetics)
    target = np.array(target)[1:].reshape(-1).tolist()
    sigma_Y = errors[1:].reshape(-1).tolist()
    rata_Y = rates[1:].reshape(-1).tolist()
    
    with mcmc_model:
        sigma = pm.HalfCauchy('sigma', beta=1, initval=0.1)
        y_obs = pm.Normal(f"rates", mu=target, sigma=sigma, observed=rata_Y, shape=len(rata_Y))
    
    return mcmc_model


In [4]:
def opt_model(dataset, k_kinetics, k_sigma_priors=0.01, draws=10000, tune=2000, chains=4, cores=4):
    mcmc_model = get_model(dataset, k_kinetics, k_sigma_priors=k_sigma_priors)
    idata = pm.sample(draws=draws,model=mcmc_model, chains=chains, cores=cores, tune=tune)
    return idata

def eval_model(idata, dataset):
    predict_ks = get_predict_ks(idata)
    predict = get_target(predict_ks, df, k_kinetics)
    rates_y  = dataset.get_rates()
    r2 = r2_loss(predict[1:],rates_y[1:])
    return r2

In [7]:
from scipy.optimize import leastsq
import time
def ltq_fit(dataset, k_kinetics):
    def _error_loss(ks, dataset):
        df = dataset.get_df()
        rates_y = dataset.get_rates()
        predict= get_target(ks, df, k_kinetics)
        # r2 = r2_loss(predict[1:],rates_y[1:])
        res =  (rates_y[1:] - predict[1:]).reshape(-1)
        return res
    
    ks_o = np.repeat(1,11).tolist()
    ks_res =leastsq(_error_loss, ks_o, args=(dataset,))[0]
    return ks_res

dataset = MyDataset("dataset/data.csv")
k_kinetics = np.repeat(1,11).astype(np.uint8).tolist()

df = dataset.get_df()
para = ltq_fit(dataset, k_kinetics)

ks_res = para
print(para)


[ 3.49616851e+00  3.55822391e-06  5.97588865e-02 -4.61461048e-07
  2.10967336e-05 -1.15460917e-06  2.98915784e-04 -7.51800478e-03
  2.38296373e-02  8.05773413e-05 -3.19028538e-04]
